In [1]:
%matplotlib inline

Source: [https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#exercise-computing-word-embeddings-continuous-bag-of-words](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#exercise-computing-word-embeddings-continuous-bag-of-words)

# Word Embeddings: Encoding Lexical Semantics

Word embeddings are dense vectors of real numbers, one per word in your
vocabulary. In NLP, it is almost always the case that your features are
words! But how should you represent a word in a computer? You could
store its ascii character representation, but that only tells you what
the word *is*, it doesn't say much about what it *means* (you might be
able to derive its part of speech from its affixes, or properties from
its capitalization, but not much). Even more, in what sense could you
combine these representations? We often want dense outputs from our
neural networks, where the inputs are $|V|$ dimensional, where
$V$ is our vocabulary, but often the outputs are only a few
dimensional (if we are only predicting a handful of labels, for
instance). How do we get from a massive dimensional space to a smaller
dimensional space?

How about instead of ascii representations, we use a one-hot encoding?
That is, we represent the word $w$ by

\begin{align}\overbrace{\left[ 0, 0, \dots, 1, \dots, 0, 0 \right]}^\text{|V| elements}\end{align}

where the 1 is in a location unique to $w$. Any other word will
have a 1 in some other location, and a 0 everywhere else.

There is an enormous drawback to this representation, besides just how
huge it is. It basically treats all words as independent entities with
no relation to each other. What we really want is some notion of
*similarity* between words. Why? Let's see an example.

Suppose we are building a language model. Suppose we have seen the
sentences

* The mathematician ran to the store.
* The physicist ran to the store.
* The mathematician solved the open problem.

in our training data. Now suppose we get a new sentence never before
seen in our training data:

* The physicist solved the open problem.

Our language model might do OK on this sentence, but wouldn't it be much
better if we could use the following two facts:

* We have seen  mathematician and physicist in the same role in a sentence. Somehow they
  have a semantic relation.
* We have seen mathematician in the same role  in this new unseen sentence
  as we are now seeing physicist.

and then infer that physicist is actually a good fit in the new unseen
sentence? This is what we mean by a notion of similarity: we mean
*semantic similarity*, not simply having similar orthographic
representations. It is a technique to combat the sparsity of linguistic
data, by connecting the dots between what we have seen and what we
haven't. This example of course relies on a fundamental linguistic
assumption: that words appearing in similar contexts are related to each
other semantically. This is called the `distributional
hypothesis <https://en.wikipedia.org/wiki/Distributional_semantics>`__.


# Getting Dense Word Embeddings

How can we solve this problem? That is, how could we actually encode
semantic similarity in words? Maybe we think up some semantic
attributes. For example, we see that both mathematicians and physicists
can run, so maybe we give these words a high score for the "is able to
run" semantic attribute. Think of some other attributes, and imagine
what you might score some common words on those attributes.

If each attribute is a dimension, then we might give each word a vector,
like this:

\begin{align}q_\text{mathematician} = \left[ \overbrace{2.3}^\text{can run},
   \overbrace{9.4}^\text{likes coffee}, \overbrace{-5.5}^\text{majored in Physics}, \dots \right]\end{align}

\begin{align}q_\text{physicist} = \left[ \overbrace{2.5}^\text{can run},
   \overbrace{9.1}^\text{likes coffee}, \overbrace{6.4}^\text{majored in Physics}, \dots \right]\end{align}

Then we can get a measure of similarity between these words by doing:

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = q_\text{physicist} \cdot q_\text{mathematician}\end{align}

Although it is more common to normalize by the lengths:

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = \frac{q_\text{physicist} \cdot q_\text{mathematician}}
   {\| q_\text{\physicist} \| \| q_\text{mathematician} \|} = \cos (\phi)\end{align}

Where $\phi$ is the angle between the two vectors. That way,
extremely similar words (words whose embeddings point in the same
direction) will have similarity 1. Extremely dissimilar words should
have similarity -1.


You can think of the sparse one-hot vectors from the beginning of this
section as a special case of these new vectors we have defined, where
each word basically has similarity 0, and we gave each word some unique
semantic attribute. These new vectors are *dense*, which is to say their
entries are (typically) non-zero.

But these new vectors are a big pain: you could think of thousands of
different semantic attributes that might be relevant to determining
similarity, and how on earth would you set the values of the different
attributes? Central to the idea of deep learning is that the neural
network learns representations of the features, rather than requiring
the programmer to design them herself. So why not just let the word
embeddings be parameters in our model, and then be updated during
training? This is exactly what we will do. We will have some *latent
semantic attributes* that the network can, in principle, learn. Note
that the word embeddings will probably not be interpretable. That is,
although with our hand-crafted vectors above we can see that
mathematicians and physicists are similar in that they both like coffee,
if we allow a neural network to learn the embeddings and see that both
mathematicians and physicists have a large value in the second
dimension, it is not clear what that means. They are similar in some
latent semantic dimension, but this probably has no interpretation to
us.


In summary, **word embeddings are a representation of the *semantics* of
a word, efficiently encoding semantic information that might be relevant
to the task at hand**. You can embed other things too: part of speech
tags, parse trees, anything! The idea of feature embeddings is central
to the field.


# Word Embeddings in Pytorch

Before we get to a worked example and an exercise, a few quick notes
about how to use embeddings in Pytorch and in deep learning programming
in general. Similar to how we defined a unique index for each word when
making one-hot vectors, we also need to define an index for each word
when using embeddings. These will be keys into a lookup table. That is,
embeddings are stored as a $|V| \times D$ matrix, where $D$
is the dimensionality of the embeddings, such that the word assigned
index $i$ has its embedding stored in the $i$'th row of the
matrix. In all of my code, the mapping from words to indices is a
dictionary named word\_to\_ix.

The module that allows you to use embeddings is torch.nn.Embedding,
which takes two arguments: the vocabulary size, and the dimensionality
of the embeddings.

To index into this table, you must use torch.LongTensor (since the
indices are integers, not floats).




In [2]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [3]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward0>)


# An Example: N-Gram Language Modeling

Recall that in an n-gram language model, given a sequence of words
$w$, we want to compute

\begin{align}P(w_i | w_{i-1}, w_{i-2}, \dots, w_{i-n+1} )\end{align}

Where $w_i$ is the ith word of the sequence.

In this example, we will compute the loss function on some training
examples and update the parameters with backpropagation.




In [4]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]
[519.1247088909149, 516.5994164943695, 514.0900070667267, 511.59886717796326, 509.1251769065857, 506.6661686897278, 504.2211420536041, 501.7876298427582, 499.365535736084, 496.95503854751587]


# Exercise: Computing Word Embeddings: Continuous Bag-of-Words

The Continuous Bag-of-Words model (CBOW) is frequently used in NLP deep
learning. It is a model that tries to predict words given the context of
a few words before and a few words after the target word. This is
distinct from language modeling, since CBOW is not sequential and does
not have to be probabilistic. Typcially, CBOW is used to quickly train
word embeddings, and these embeddings are used to initialize the
embeddings of some more complicated model. Usually, this is referred to
as *pretraining embeddings*. It almost always helps performance a couple
of percent.

The CBOW model is as follows. Given a target word $w_i$ and an
$N$ context window on each side, $w_{i-1}, \dots, w_{i-N}$
and $w_{i+1}, \dots, w_{i+N}$, referring to all context words
collectively as $C$, CBOW tries to minimize

\begin{align}-\log p(w_i | C) = -\log \text{Softmax}(A(\sum_{w \in C} q_w) + b)\end{align}

where $q_w$ is the embedding for word $w$.

Implement this model in Pytorch by filling in the class below. Some
tips:

* Think about which parameters you need to define.
* Make sure you know what shape each operation expects. Use .view() if you need to
  reshape.




In [5]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMBEDDING_DIM = 100

# reding the dataset
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])


class CBOW(torch.nn.Module):

    def __init__(self, vocab_size, embedding_dim):

        super(CBOW, self).__init__()

        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.activation_function1 = nn.ReLU()

        self.linear2 = nn.Linear(128, vocab_size)
        self.activation_function2 = nn.LogSoftmax(dim = -1)

    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

model = CBOW(vocab_size, EMBEDDING_DIM)

losses = []
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

#training the model
for epoch in range(10):
  total_loss = 0

  for context, target in data:
    # Step 1. Prepare the inputs to be passed to the model
    context_vec = make_context_vector(context, word_to_ix)

    # Step 2. Recall that torch *accumulates* gradients.
    model.zero_grad()

    # Step 3. Run the forward pass, getting log probabilities over next words
    log_probs = model(context_vec)

    #step 4.Compute the loss function.
    loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

    # Step 5. Do the backward pass and update the gradient
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    # Get the Python number from a 1-element Tensor by calling tensor.item()
    total_loss += loss.item()

  losses.append(total_loss)

print(losses)

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]
[466.3224849700928, 448.71989250183105, 432.0826778411865, 416.12587881088257, 400.73743295669556, 385.838103055954, 371.44805550575256, 357.52513909339905, 343.9932096004486, 330.7795649766922]


# part 1 Train your CBOW embeddings for both datasets

**CBOW2 with a context width of 2 (in both directions) for the Hotel Reviews dataset.**


In [2]:
!gdown 1BQOySY3HmDvVbNXLSlh59XxIvLKtjZyU
!gdown 1Auxk1RsQgRSzAjVIeXPFwFAKIbkNrWmT

Downloading...
From: https://drive.google.com/uc?id=1BQOySY3HmDvVbNXLSlh59XxIvLKtjZyU
To: /content/scifi.txt
100% 86.1M/86.1M [00:00<00:00, 108MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1Auxk1RsQgRSzAjVIeXPFwFAKIbkNrWmT
To: /content/tripadvisor_hotel_reviews.csv
100% 15.0M/15.0M [00:00<00:00, 121MB/s] 


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import csv
import string
import re
#import nltk

In [3]:
# define the hyperparameters
#CBOW2 with a context width of 2 (in both directions) for the Hotel Reviews dataset.

CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMBEDDING_DIM = 50

In [4]:
# preprocessing the text
def preprocess_text(text):
    # remove special characters
    #text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'[^\w\s]', '', text)
    # lowercasing
    text = text.lower()
    # tokenization
    tokens = text.split()
    # remove stop words
    #stop_words = set(stopwords.words('english'))
    #filtered_words = [word for word in words if word not in stop_words]
    #processed_text = ' '.join(words)
    return tokens

# Reading the csv and txt file and preprocessing the text
def read_data(file_path, is_csv=False):
    if is_csv:
        # reading a csv file
        data = pd.read_csv(file_path)
        raw_text = data['Review'].str.cat(sep=' ')
    else:
        # reading a txt file
        with open(file_path, 'r') as file:
            raw_text = file.read()
    preprocessed_text = preprocess_text(raw_text)
    return preprocessed_text

In [5]:
# build a vocabulary
def build_vocab(preprocessed_text):
    vocab = set(preprocessed_text)
    vocab_size = len(vocab)
    word_to_ix = {word: i for i, word in enumerate(vocab)}
    return word_to_ix, vocab_size

# creat target and context data， width of 2
def create_data(preprocessed_text):
    data = []
    for i in range(2, len(preprocessed_text) - 2):
        context = [preprocessed_text[i -2 ], preprocessed_text[i - 1], preprocessed_text[i + 1], preprocessed_text[i + 2]]
        target = preprocessed_text[i]
        data.append((context, target))
    return data

In [6]:
CONTEXT_SIZE = 2

# define the CBOW model
class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim * CONTEXT_SIZE * 2, 128)
        self.activation_function1 = nn.ReLU()
        self.linear2 = nn.Linear(128, vocab_size)
        self.activation_function2 = nn.LogSoftmax(dim=-1)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view(-1, EMBEDDING_DIM * CONTEXT_SIZE * 2)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

# make context words to vectors/index
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

In [7]:
# reading the file
file_path = "tripadvisor_hotel_reviews.csv"
is_csv = True
preprocessed_text = read_data(file_path, is_csv)

In [8]:
# making a vocabulary
word_to_ix, vocab_size = build_vocab(preprocessed_text)

In [9]:
# creating context and target data
data = create_data(preprocessed_text)

In [10]:
import nltk
from collections import Counter
from nltk.corpus import wordnet as wn

#all_tokens = [token for review in df['tokenized_reviews'] for token in review]
word_freq = Counter(preprocessed_text)

# Sort words by frequency
sorted_words = [word for word, freq in word_freq.most_common()]

# Use nltk's part-of-speech tagger
nltk.download('averaged_perceptron_tagger')
tags = nltk.pos_tag(sorted_words)

# Filter nouns, verbs, and adjectives
nouns = [word for word, tag in tags if tag.startswith('NN')]
verbs = [word for word, tag in tags if tag.startswith('VB')]
adjectives = [word for word, tag in tags if tag.startswith('JJ')]

# Select frequent and rare words
frequent_nouns = nouns[:10]
rare_nouns = nouns[-10:]

frequent_verbs = verbs[:10]
rare_verbs = verbs[-10:]

frequent_adj = adjectives[:10]
rare_adj = adjectives[-10:]

print(f"Frequent Nouns: {frequent_nouns}")
print(f"Rare Nouns: {rare_nouns}")
print(f"Frequent Verbs: {frequent_verbs}")
print(f"Rare Verbs: {rare_verbs}")
print(f"Frequent Adjectives: {frequent_adj}")
print(f"Rare Adjectives: {rare_adj}")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Frequent Nouns: ['hotel', 'room', 'staff', 'stay', 'rooms', 'location', 'service', 'time', 'beach', 'night']
Rare Nouns: ['documentto', 'hardline', 'opinionin', 'mossos', 'desquadra', 'undercover', 'tallers', 'muggings', 'cornerone', 'swears']
Frequent Verbs: ['did', 'stayed', 'breakfast', 'got', 'bed', 'went', 'want', 'make', 'say', 'lobby']
Rare Verbs: ['esteem', 'restaurantrooms', 'familiesrecommend', 'pilling', 'fingerprinted', 'indemnified', 'blackmailed', 'attests', 'barcelonins', 'joball']
Frequent Adjectives: ['great', 'good', 'nice', 'clean', 'place', 'small', 'little', 'walk', 'excellent', 'best']
Rare Adjectives: ['uwajimaya', 'roomcafe', 'aerate', 'uplift', 'gobcn', 'invalid', 'unbolted', 'elusive', 'evasive', 'victimizedthis']


In [15]:
from torch.utils.data import Dataset, DataLoader

class CBOWDataset(Dataset):
    def __init__(self, data, word_to_ix):
        self.data = data
        self.word_to_ix = word_to_ix

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        context, target = self.data[idx]
        context_vec = make_context_vector(context, self.word_to_ix)
        target_tensor = torch.tensor([word_to_ix[target]], dtype=torch.long)
        return context_vec, target_tensor

# Create an instance of CBOWDataset and DataLoader
dataset = CBOWDataset(data, word_to_ix)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
# initialize the cbow model
model = CBOW(vocab_size, EMBEDDING_DIM).to(device)

# define the loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1)

print(len(data))

# training the model
for epoch in range(13):
    total_loss = 0
    for context_vecs, target_tensors in dataloader:
        context_vecs, target_tensors = context_vecs.to(device), target_tensors.to(device)

        model.zero_grad()
        log_probs = model(context_vecs)
        log_probs = log_probs.view(-1, vocab_size)
        target_tensors = target_tensors.view(-1)

        loss = loss_function(log_probs, target_tensors.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {total_loss}")

2138758
Epoch 1, Loss: 7463899.343604088
Epoch 2, Loss: 1937015.7776956558
Epoch 3, Loss: 408139.58357048035
Epoch 4, Loss: 565074.1239671707
Epoch 5, Loss: 362564.73153972626
Epoch 6, Loss: 339762.54636859894
Epoch 7, Loss: 339722.0997514725
Epoch 8, Loss: 339711.8210282326
Epoch 9, Loss: 339700.80089235306
Epoch 10, Loss: 339767.42577934265
Epoch 11, Loss: 339736.4159064293
Epoch 12, Loss: 339750.90731954575
Epoch 13, Loss: 339763.3398504257


In [19]:
# Save the model
model_path = "cbow_model_2.pth"
torch.save(model.state_dict(), model_path)

2.2 For each of the 9 chosen words, retrieve the 5 closest words according to your trained CBOW2 model. List them in your report and comment on the performance of your model: do the neighbors the model provides make sense? Discuss.

nouns: staff (frequent), location (frequent); mossos (rare)

verbs: stayed (frequent), got (frequent); esteemed (rare)

adjective: clean (frequent), small (frequent); aerate (rare)


In [20]:
import torch.nn as nn

def get_closest_word(word, topn=5):
    word_distance = []
    emb = model.embeddings
    device = emb.weight.device  # Determine the device of the model
    pdist = nn.PairwiseDistance().to(device)
    i = word_to_ix[word]
    lookup_tensor_i = torch.tensor([i], dtype=torch.long).to(device)  # Move tensor to the same device
    v_i = emb(lookup_tensor_i)
    index_to_word = {a: word for word, a in word_to_ix.items()}
    for j in range(vocab_size):
        if j != i:
            lookup_tensor_j = torch.tensor([j], dtype=torch.long).to(device)  # Move tensor to the same device
            v_j = emb(lookup_tensor_j)
            word_distance.append((index_to_word[j], float(pdist(v_i, v_j))))
    word_distance.sort(key=lambda x: x[1])
    return word_distance[:topn]

In [21]:
words = {'staff', 'location', 'mossos', 'stayed', 'got', 'esteemed', 'clean', 'small', 'aerate', 'good'}

for word in words:
  closest_word = get_closest_word(word, topn=5)
  print(f"Words closest to {word}:{closest_word}")

Words closest to stayed:[('desert', 141.33218383789062), ('tyrip', 146.5635223388672), ('sported', 151.49803161621094), ('shells', 152.20143127441406), ('fore', 152.7379150390625)]
Words closest to esteemed:[('styleoverall', 6.458023548126221), ('griping', 6.484110355377197), ('wellas', 6.545387268066406), ('awrsome', 6.59826135635376), ('areasto', 6.606688022613525)]
Words closest to aerate:[('70night', 6.395145893096924), ('spaingetting', 6.5656657218933105), ('barcelonas', 6.713163375854492), ('bedsbedding', 6.819931983947754), ('nats', 6.839423656463623)]
Words closest to mossos:[('staffagain', 63.37111282348633), ('enterprise', 64.01919555664062), ('support', 86.26815795898438), ('reborn', 86.96928405761719), ('hayley', 87.17584991455078)]
Words closest to got:[('elephant', 233.0531768798828), ('sandy', 242.52674865722656), ('awoke', 244.12974548339844), ('multiple', 245.31024169921875), ('brazil', 245.5233612060547)]
Words closest to good:[('smile', 207.35301208496094), ('pants',